# 日本語クリーナ for CSV

## 事前準備

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import re

### 入力ファイルと出力ファイルの指定

In [2]:
input_file = "test_data.csv" # 入力ファイル名
output_file = "processed_oneline"   # 出力ファイル名
split_oneliner_dir = "split-oneliner-files" # 行ごとに分割したファイルを保存するディレクトリ
col_q = "質問" # 分割する対象の列名
col_a = "回答" # 分割する対象の列名

### ファイルの読み込み

In [3]:
# ファイルパスを指定してCSVファイルを読み込む。読み込むときはutf8でエンコードする。改行コードは\nに統一する。
df = pd.read_csv(input_file, encoding="utf8", lineterminator="\n")
# 必要な列だけを抽出する
df = df[[col_q, col_a]]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   質問      67 non-null     object
 1   回答      67 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


### カスタム関数

各種データに対して以下のセルは修正すること

In [4]:
# 処理例
# - 変換前: "これはテストです。\n次の行の内容です。"
# - 変換後: "これはテストに関する質問です。\n次の行の内容です。"
def surround_first_line(text):
    lines = text.split('\n')
    if len(lines) > 1:  # 複数行の場合
        if lines[0].endswith('。'): # 1行目が"。"で終わる場合
            lines[0] = lines[0][:-1]  # 最後の文字 "。" を削除
        if len(lines[0]) > 0:   # 1行目の文字数が0より大きい場合
            lines[0] = f"{lines[0]}に関する質問です。"
    return '\n'.join(lines)

# 例
# - 変換前: "この文には句点がありません"
# - 変換後: "この文には句点がありません。"
def surround_last_line(text):
    if text[-1] not in ['。', '？', '）', '】', '?', '!']:
        return f"{text}。"
    else:
        return text
    

# 空行とスペースを削除する関数
def remove_empty_lines_and_spaces(text):
    text = re.sub(r'[A-Z][0-9]{4}', '', text)
    text = re.sub(r'[A-Z][0-9]{3}', '', text)
    text = re.sub(r'\n\s*\n', '\n', text)  # 空行を削除
    text = text.replace(' ', '')  # 半角スペースを削除
    text = text.replace('　', '')  # 全角スペースを削除
    text = text.replace('"', '')
    text = re.sub(r'[\r\n]+', '', text)  # 改行コードを削除
    return text

### カスタム処理

In [5]:
# df[col_q]の1行目に"に関する質問です。"を追加
df[col_q] = df[col_q].apply(surround_first_line)

# df[col_q]の末尾が "）" もしくは "。" もしくは"？" で終わっていないものに関して "。これに関する情報を知りたい。"を追加
df[col_q] = df[col_q].apply(surround_last_line)

### 正規表現と置換のみで処理

In [6]:
# df[col_q]の末尾が "か。"で終わるものを"か？"に変換
df[col_q] = df[col_q].str.replace('か。', 'か？', regex=True)

# df[col_q]の末尾が "は。"で終わるものを"は？"に変換
df[col_q] = df[col_q].str.replace('は。$', 'は？', regex=True)

# df[col_q]の末尾が"か" で終わるものを"か？"に変換
df[col_q] = df[col_q].str.replace('ない。$', 'ない。どうすればよいか？', regex=True)

# df[col_q]の末尾が"したい。" で終わるものを"したい。どうすれば良いか？"に変換
df[col_q] = df[col_q].str.replace('したい。$', 'したい。どうすれば良いか？', regex=True)

# df[col_q]の末尾が"方法" で終わるものを"方法は？"に変換
df[col_q] = df[col_q].str.replace('方法$', '方法を教えてほしい。', regex=True)

# df[col_q]の末尾が"方法" で終わるものを"方法は？"に変換
df[col_q] = df[col_q].str.replace('された。$', 'された。どうすれば良いか？', regex=True)

# df[col_q]の末尾が"方法" で終わるものを"方法は？"に変換
df[col_q] = df[col_q].str.replace('ついて$', 'ついて教えてほしい。', regex=True)

In [7]:
# 全てのセルに対して空行を削除
df = df.applymap(remove_empty_lines_and_spaces)

## ファイルの保存

### onelinerのcsvとして保存(2列)

In [8]:
df.to_csv(f'{output_file}.csv', index=False)

### onelinerのtxtとして保存

In [9]:
# 質問と回答を結合してテキストデータを作成
merged_text = df[col_q] + df[col_a]
# テキストファイルに保存
with open(f'{output_file}.txt', 'w') as file:
    for text in merged_text:
        file.write(text + "\n")

### onelinerのtxtとしてoutputディレクトリに分割保存

In [10]:
# outputディレクトリを削除する
output_dir = split_oneliner_dir
if os.path.exists(f'{output_dir}'):
    shutil.rmtree(f'{output_dir}')

# (1) outputディレクトリを作成する
os.makedirs(f'{output_dir}')

# (2) df[col_q]とdf[col_a]を結合して1行1行分割し、outputディレクトリに保存する
for index, val in df.iterrows():
    question = val[col_q]
    answer = val[col_a]
    combined_text = question + answer
    # インデックスを4桁のゼロパディングに変換
    padded_index = str(index).zfill(4)
    # 保存するファイル名を作成
    filename = os.path.join(output_dir, f"{output_file}-{padded_index}.txt")
    
    # ファイルを書き込む前にディレクトリが存在するかを確認
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # ファイルを保存
    with open(filename, 'w') as file:
        file.write(combined_text)

# 分割されたファイルをtgz形式で圧縮する
shutil.make_archive(f'split-{output_file}', 'gztar', output_dir)

# outputディレクトリを削除する
shutil.rmtree(f'{output_dir}')